In [275]:
import logging
from itertools import combinations
import pandas as pd
import numpy as np
from geopy.distance import geodesic
import geopy
import networkx as nx
import random
import functools
from icecream import ic

logging.basicConfig(level=logging.DEBUG)

In [276]:
def counter(fn):
    """Simple decorator for counting number of calls"""

    @functools.wraps(fn)
    def helper(*args, **kargs):
        helper.calls += 1
        return fn(*args, **kargs)

    helper.calls = 0
    return helper


@counter
def tsp_cost(tsp):
    assert tsp[0] == tsp[-1] #this is a cycle
    assert set(tsp) == set(range(len(cities)))
    tot_cost = 0
    for c1, c2 in zip(tsp, tsp[1:]):
        tot_cost += dist_matrix[c1, c2]
    return tot_cost
        

In [277]:
cities_selection = 3 ##PARAMETER SELECTION
if cities_selection == 0:
    cities = pd.read_csv('cities/china.csv', header=None, names=['name', 'lat', 'lon'])
elif cities_selection == 1:
    cities = pd.read_csv('cities/italy.csv', header=None, names=['name', 'lat', 'lon'])
elif cities_selection == 2:
    cities = pd.read_csv('cities/russia.csv', header=None, names=['name', 'lat', 'lon'])
elif cities_selection == 3:
    cities = pd.read_csv('cities/us.csv', header=None, names=['name', 'lat', 'lon'])
elif cities_selection == 4:
    cities = pd.read_csv('cities/vanuatu.csv', header=None, names=['name', 'lat', 'lon'])
    
    
dist_matrix = np.zeros((len(cities), len(cities)))
for c1, c2 in combinations(cities.itertuples(), 2):
    dist_matrix[c1.Index, c2.Index] = dist_matrix[c2.Index, c1.Index] = geopy.distance.geodesic(
        (c1.lat, c1.lon), (c2.lat, c2.lon)
    ).km


Initialization phase

In [278]:
def greedy_approach(starting_index):
    n = len(dist_matrix)
    unvisited = set(range(n))
    tour = [starting_index]  # Start from the first city
    unvisited.remove(starting_index)
        
    while unvisited:
        last = tour[-1]
        next_city = min(unvisited, key=lambda city: dist_matrix[last][city])
        tour.append(next_city)
        unvisited.remove(next_city)

    tour.append(starting_index)
    return tour
    

In [279]:
def random_init():
    tour = list(range(len(cities)))
    random.shuffle(tour)
    tour.append(tour[0])
    return tour
    

In [280]:
select_initialization = 2 ##PARAMETER SELECTION
if select_initialization == 1:
    tour = greedy_approach(0)
elif select_initialization == 2:
    tour = random_init()
else:
    print("Not valid initialization")
    

In [281]:
ordered_cities = cities.iloc[tour]
print(ordered_cities)
print(f"Found a path of {len(tour)-1} steps, total length {tsp_cost(tour):.2f}km")

                 name        lat         lon
48         Chesapeake  36.679376  -76.301788
214          Palm Bay  27.985624  -80.662621
118           Hayward  37.628139 -122.106341
157       Little Rock  34.725432  -92.358556
20          Baltimore  39.300213  -76.610516
..                ...        ...         ...
323     Winston‐Salem  36.103262  -80.260578
224        Pittsburgh  40.439752  -79.976592
198  North Charleston  32.917342  -80.065150
203         Oceanside  33.224572 -117.306227
48         Chesapeake  36.679376  -76.301788

[327 rows x 3 columns]
Found a path of 326 steps, total length 648087.73km


Evolutionary algorithm

In [282]:
def initialize_population(num_cities):
    max_population = min(num_cities * 2, 100) 
    population = []

    for i in range(max_population):
        start_index = i % num_cities  
        tour = greedy_approach(start_index)
        population.append(tour)

    return population

def tournament_selection(population, k=10):
    selected_indices = random.sample(list(range(len(population))), k)
    selected_tours = [population[i] for i in selected_indices]
    return min(selected_tours, key=tsp_cost)

def mutate(tour, num_swaps = 4):
    n = len(tour)
    for _ in range(num_swaps):
        a, b = random.sample(range(1, n - 1), 2)
        tour[a], tour[b] = tour[b], tour[a]
    return tour

def local_search(path):
    best = path
    improved = True
    while improved:
        improved = False
        for i in range(1, len(path) - 2):
            for j in range(i + 1, len(path) - 1):
                if j - i == 1: continue  # Skip adjacent cities
                new_path = best[:]
                new_path[i:j + 1] = reversed(best[i:j + 1])  # 2-opt swap
                if tsp_cost(new_path) < tsp_cost(best):
                    best = new_path
                    improved = True
    return best

def inv_crossover(parent1, parent2):
    child = parent1.copy()
    n = len(parent1) - 1 
    start = random.randint(1, n - 1)  
    
    start_value = parent1[start]
    index_in_parent2 = parent2.index(start_value)
    
    if start < index_in_parent2:
        segment_to_invert = parent1[start:index_in_parent2 + 1]
    else:
        segment_to_invert = parent1[index_in_parent2:start + 1]
    
    segment_to_invert.reverse()
    
    if start < index_in_parent2:
        child[start:index_in_parent2 + 1] = segment_to_invert
    else:
        child[index_in_parent2:start + 1] = segment_to_invert

    child[0] = child[-1] = child[0]
    
    return child


In [283]:
def mutation_algorithm(population, iterations=10000, initial_mutation_rate=0.6, num_swapping=6, cooling_rate=0.99):
    best_population = population
    best_fitness = tsp_cost(best_population)
    mutation_rate = initial_mutation_rate
    cooling_rate = 0.80

    for _ in range(iterations):
        new_population = best_population.copy()
        if random.random() < mutation_rate:
            new_population = mutate(new_population, num_swapping)
            new_population = local_search(new_population)
        
        new_fitness = tsp_cost(new_population)
    
        if new_fitness < best_fitness:
            best_population = new_population
            best_fitness = new_fitness
            # Decrease num_swapping if we're improving. In this way I improve exploitation if the solution is better
            num_swapping = max(1, num_swapping - 2)
        else:
            mutation_rate = min(1.0, mutation_rate*cooling_rate + abs(new_fitness - best_fitness) / 1000)
            # Increase num_swapping if not improving, encouraging more exploration.
            num_swapping = min(len(best_population) // 2, num_swapping + 1)
        
        # Decay the mutation rate over time. With a value equal to 0.80, the mutation rate is decreased of 20%
        mutation_rate *= cooling_rate

    return best_population


In [284]:
def crossover_algorithm(gen_number, path, mutation_rate=0.5):
    population = [path]
    population.extend(initialize_population(len(cities)))
    
    for generation in range(gen_number):
        next_generation = []
        
        # Create new offspring
        while len(next_generation) < len(population):
            # Parent selection
            parent1 = tournament_selection(population)
            parent2 = tournament_selection(population)
            
            # Perform crossover
            child = inv_crossover(parent1, parent2)
            
            # Optional mutation to maintain diversity
            if random.random() < mutation_rate:
                child = mutate(child)
            
            # Only add child if it improves fitness or meets criteria
            if tsp_cost(child) < max(tsp_cost(parent1), tsp_cost(parent2)):
                next_generation.append(child)
            else:
                next_generation.append(random.choice([parent1, parent2]))  # Retain one parent if no improvement
        
        # Update population
        population = next_generation
        
        # Optionally print progress
        if generation % 10 == 0:
            best_fitness = min(tsp_cost(tour) for tour in population)
            print(f"Generation {generation}, Best Fitness: {best_fitness}")

    # Return the best solution found
    best_tour = min(population, key=tsp_cost)
    return best_tour


In [285]:
population = tour.copy()

algorithm_selection = 2 ##PARAMETER SELECTION
if algorithm_selection == 1:
    population = mutation_algorithm(population)
elif algorithm_selection == 2:
    population = crossover_algorithm(100, population)
else:
    print("Not valid algorithm")

ordered_cities = cities.iloc[population]
print(ordered_cities)
print(f"Found a path of {len(population)-1} steps, total length {tsp_cost(population):.2f}km, number of calls {tsp_cost.calls}")

Generation 0, Best Fitness: 46304.14348276868
Generation 10, Best Fitness: 46304.14348276868
Generation 20, Best Fitness: 46304.14348276868
Generation 30, Best Fitness: 46304.14348276868
Generation 40, Best Fitness: 46304.14348276868
Generation 50, Best Fitness: 46304.14348276868
Generation 60, Best Fitness: 46304.14348276868
Generation 70, Best Fitness: 46304.14348276868
Generation 80, Best Fitness: 46304.14348276868
Generation 90, Best Fitness: 46304.14348276868
                 name        lat         lon
58   Colorado Springs  38.867255 -104.760749
232            Pueblo  38.273147 -104.612378
121   Highlands Ranch  39.540810 -104.971033
43         Centennial  39.590568 -104.869118
73             Denver  39.761849 -104.880625
..                ...        ...         ...
269        Santa Rosa  38.446816 -122.706136
268       Santa Maria  34.933214 -120.443815
9           Anchorage  61.177549 -149.274354
125          Honolulu  21.324347 -157.847640
58   Colorado Springs  38.867255 -10